Итоговый проект.

Реализовать графическую программу с использованием библиотеки Tkinter, которая состоит из поля ввода текста, раскрывающегося списка и полем вывода надписи и кнопки.

Логика работы программы:

1.Пользователь вводит последовательность чисел через запятую

2.Выбирает один из вариантов сортировки

3.После нажатия на кнопку Start происходит сортировка последовательности с последующим выводом в текстовое поле вывода

4.Реализовать вывод времени затраченного на сортировку



Код должен содержать комментарии, а также все необходимые проверки на исключения. 

Код должен быть максимально читаем. При написании следует прибегнуть к стандартным библиотекам тестирования!

In [2]:
import time
from typing import List, Union, Callable, Any
import tkinter as tk
from tkinter import ttk
from tkinter.scrolledtext import ScrolledText


def is_real_num(val: str) -> bool:
    """
    Проверяет, является ли строка вещественным числом
    :val: str
    :return: bool
    """
    try:
        float(val)
        return True
    except ValueError:
        return False


def str_to_num(val: str) -> Union[int, float]:
    """
    Преобразует значение строки в число соответствующего ему типа.
    Вернет ошибку, если в строке содержится нечисловое значение.
    :val: str
    :return: int|float
    """
    if val.strip().isdecimal():
        return int(val)
    else:
        return float(val)


def str_to_num_list(line: str) -> List[Union[int, float]]:
    """
    Преобразует строку с числами, разделенными запятыми, в список с числами
    :line: str
    :return: list[int | float]
    """
    return [str_to_num(i) for i in line.split(',') if is_real_num(i)]


def bubble_sort(lst: List[Union[int, float]]) -> bool:
    """
    Функция пузырьковой сортировки (меняет исходный список)
    :lst: list[int | float]
    :return: bool
    """
    steps = 0
    # циклично перемещаем наибольший элемент к правой границе списка
    # при достижение правой границы смещаем ее на 1 элемент влево
    # и двигаем очередной наибольший элемент уже к ней
    for last_elem_idx in range(len(lst) - 1, 0, -1):
        for idx in range(last_elem_idx):
            steps += 1
            if lst[idx] > lst[idx + 1]:
                temp = lst[idx]
                lst[idx] = lst[idx + 1]
                lst[idx + 1] = temp
    return True


def heapify(lst: List[Union[int, float]], n: int, i: int) -> bool:
    """
    функция преобразования в двоичную кучу, где lst - список, i - корневой индекс и n - размер кучи
    (меняет исходный список)
    :lst: list[int | float]
    :n: int
    :i: int
    :return: bool
    """
    largest = i
    left = 2 * i + 1
    right = 2 * i + 2
    if left < n and lst[i] < lst[left]:  # проверка существования левого элемента, который больше корня
        largest = left
    if right < n and lst[largest] < lst[right]:  # аналогичная проверка для правого элемента.
        largest = right
    if largest != i:
        lst[i], lst[largest] = lst[largest], lst[i]  # меняем корень если это требуется
        try:
            res = heapify(lst, n, largest)  # применяем функцию к корневому элементу
        except RecursionError:
            return False
        if not res:
            return False
    return True


def heap_sort(lst: List[Union[int, float]]) -> bool:
    """
    Функция пирамидальной сортировки (меняет исходный список). Вызывает вспомогательную функцию heapify()
    :lst: list[int | float]
    :return: bool
    """
    n = len(lst)
    for i in range(n, -1, -1):
        res1 = heapify(lst, n, i)  # на данном этапе список еще не отсортирован,
        if not res1:
            return False
    for i in range(n - 1, 0, -1):
        # меняем 0-ой элемент с последним элементом кучи
        lst[i], lst[0] = lst[0], lst[i]
        # теперь последний элемент кучи считается отсортированным
        # и в сортировке он участвовать не будет
        res2 = heapify(lst, i, 0)
        if not res2:
            return False
    return True


# функция сортировки слиянием
def merge_sort(lst: List[Union[int, float]]) -> bool:
    """
    Функция сортировки слиянием (меняет исходный список).
    :lst: list[int | float]
    :return: bool
    """
    if len(lst) > 1:
        mid = len(lst) // 2  # делим список пополам
        left = lst[:mid]
        right = lst[mid:]
        try:
            chk1 = merge_sort(left)  # применяем функцию к левой и правой частям
            chk2 = merge_sort(right)
        except RecursionError:
            return False
        # если хоть из одной вложенной функции получен False, то возвращаем False
        if not chk1 or not chk2:
            return False
        else:
            l_idx = 0  # индекс левой половины списка
            r_idx = 0  # индекс правой половины списка
            lst_idx = 0
            # пока в левой и правой половинах остались не сверенные элементы
            while l_idx < len(left) and r_idx < len(right):
                # сверяем не записанные элементы левой и правой половин
                # записываем в объединенный список наименьший элемент и исключаем его из сравнения
                if left[l_idx] < right[r_idx]:
                    lst[lst_idx] = left[l_idx]
                    l_idx = l_idx + 1
                else:
                    lst[lst_idx] = right[r_idx]
                    r_idx = r_idx + 1
                lst_idx = lst_idx + 1
            # пока в левой половине остались не проверенные элеметнты
            while l_idx < len(left):
                # т.к. половины уже были рекурсивно отсортированы,
                # то просто допишем оставшиеся элементы в объединенный  список
                lst[lst_idx] = left[l_idx]
                l_idx = l_idx + 1
                lst_idx = lst_idx + 1
            # пока в правой половине остались непроверенные элеметнты
            while r_idx < len(right):
                # т.к. половины уже были рекурсивно отсортированы,
                # то просто допишем оставшиеся элементы в объеидинный список
                lst[lst_idx] = right[r_idx]
                r_idx = r_idx + 1
                lst_idx = lst_idx + 1
    return True


def quicksort(lst: List[Union[int, float]]) -> Union[None, List[Union[int, float]]]:
    """
    Функция быстрой сортировки.
    :lst: list[int | float]
    :return: list[int | float] | None
    """
    # Если длина списка менее двух,
    # то он является отсортированным
    if len(lst) < 2:
        return lst
    else:
        # Опорный элемент
        pivot = lst[0]
        less = [i for i in lst[1:] if i <= pivot]  # подсписок элементов меньше опорного
        greater = [i for i in lst[1:] if i > pivot]  # подсписок элементов больше опорного
        try:
            return quicksort(less) + [pivot] + quicksort(greater)
        except RecursionError:
            return None


def quicksort_shell(lst):
    """
    Функция быстрой сортировки (меняет исходный список).
    Вызывает вспомогательную функцию quicksort_shell()
    :lst: list
    :return: list
    """
    res = quicksort(lst)
    for i in range(len(res)):
        lst[i] = res[i]
    if res is not None:
        return True
    else:
        return False


def exec_time(func: Callable[[...], bool], *func_args: Any,
              **func_qwargs: Any) -> Union[float, None]:
    """
    Функция для вычисления времени, затраченного на выполнение
    переданной в качестве аргумента функции
    :func: Callable[[...], None]
    :*func_args: Any
    :**func_qwargs: Any
    :return: float | None
    """
    start_time = time.perf_counter()
    chk = func(*func_args, **func_qwargs)
    if chk:
        return (time.perf_counter() - start_time) * 1000
    else:
        return None


def main():
    """
    Тело программы
    Выполняет построение графического окна
    :return: None
    """

    # библиотека с указателеми на функции сортировок
    sort_methods = {"Пузырьковая сортировка": bubble_sort,
                    "Пирамидальная сортировка": heap_sort,
                    "Сортировка слиянием": merge_sort,
                    "Быстрая сортировка": quicksort_shell}

    # список с наименованиями сортировок (нужно для работы раскрывающегося списка)
    sort_methods_list = list(sort_methods)
    
    window = tk.Tk()  # создаем окно
    window.title("Сортировка чисел")  # указываем заголовок окна
    user_input_frame = tk.Frame(window)  # фрейм для верхней части окна
    input_frame = tk.Frame(user_input_frame)  # фрейм для ввода текста
    combo_frame = tk.Frame(user_input_frame)  # фрейм для выпадающего списка
    btn_frame = tk.Frame(user_input_frame)  # фрейм для кнопки
    text_frame = tk.Frame(window)  # фрейм для вывода результата сортировки

    # Создаем метку с поясняющим текстом для ввода текста
    input_lb = tk.Label(input_frame, width=26, anchor='w',
                        text="Введите числа через запятую: ")
    # Создаем поле ввода
    field_input = tk.Entry(input_frame, width=50)
    # Добавляем виджеты на фрейм input_frame
    input_lb.pack(side=tk.LEFT)
    field_input.pack(side=tk.LEFT)

    # Создаем метку с поясняющим текстом для выпадающего списка
    combo_lb = tk.Label(combo_frame, width=26, anchor='w',
                        text="Выберите метод сортировки: ")
    # Создаем выпадающий список
    # по умолчанию будет выбран первый элемент из sort_methods_list
    selected = tk.StringVar(value=sort_methods_list[0])
    combo = ttk.Combobox(combo_frame, width=47, textvariable=selected, 
                         values=sort_methods_list, state="readonly")
    # Добавляем виджеты на фрейм combo_frame
    combo_lb.pack(side=tk.LEFT)
    combo.pack(side=tk.LEFT)

    def start_sort():
        """
        Функция для запуска сортировки.
        Выведет рузультат в текстовое поле
        :return: None
        """
        # считываем из поля данные, введенные пользователем
        line = field_input.get()
        # разбиваем строку на элеметы по разделителю ",",
        # отбрасывем элементы, содержащие нечисловые значения,
        # преобразуем оставшиеся строковые элементы в числа
        num_list = str_to_num_list(line)
        # считываем выбор метода сортировки
        choice = combo.get()
        if len(num_list) == 0:
            text = 'Пожалуйста, введите числа для сортировки'
        else:
            # посчитает время сортировки
            # функция, хранящаяся в sort_methods[choice],
            # отсортирует значения и изменит исходный список
            time_result = exec_time(sort_methods[choice], num_list)
            text = choice
            if time_result is None:
                text += 'Сортировка прервана - достигнута максимальная глубина рекурсии'
            else:
                text += f'\nЗатраченное время: {round(time_result, 3)} мс.'
                text += '\n' + str(num_list).strip('[]')
        # разрешаем изменение текста в поле вывода
        textbox.config(state=tk.NORMAL)
        # очищаем поле от имеющегося текста
        textbox.delete('1.0', tk.END)
        # вставляем текст в поле
        textbox.insert('1.0', text)
        # блокиеруем изменение текста в поле
        textbox.config(state=tk.DISABLED)

    # поле для вывода результата
    textbox = ScrolledText(text_frame, width=60, height=20,
                           wrap=tk.WORD, state=tk.DISABLED)
    textbox.pack(fill=tk.BOTH, expand=True)

    # добавляем кнопку запуска  сортировки
    btn = tk.Button(btn_frame, text="Сортировать", command=start_sort)
    btn.pack(side=tk.TOP, pady=5)

    # располагаем фреймы на фрейме user_input_frame
    input_frame.pack(side=tk.TOP, anchor='w', pady=5)
    combo_frame.pack(side=tk.TOP, anchor='w', pady=5)
    btn_frame.pack(side=tk.TOP)
    # добавляем фреймы на окно
    user_input_frame.pack(anchor='w', padx=10, pady=5)
    text_frame.pack(side=tk.TOP, expand=True, anchor='w', padx=10, pady=5)

    # запрещаем масштабирование экрана
    window.resizable(width=False, height=False)
    # перемещаем окно на передний план
    window.lift()
    window.call('wm', 'attributes', '.', '-topmost', True)
    window.after_idle(window.call, 'wm', 'attributes', '.', '-topmost', False)
    # Цикл обработки событий окна
    window.mainloop()


if __name__ == '__main__':
    test_str = 'ac4, 8, +5, 9, 41,1,10'
    test_list = [8, 5, 9, 41, 1, 10]
    sorted_list = [1, 5, 8, 9, 10, 41]

    # тест преобразования строки в список
    assert str_to_num_list(test_str) == test_list

    # функции сортировки меняют исходный список
    # тест пузырьковой сортировки
    temp_list = test_list.copy()
    bubble_sort(temp_list)
    assert temp_list == sorted_list

    # тест пирамидальной сортировки
    temp_list = test_list.copy()
    heap_sort(temp_list)
    assert temp_list == sorted_list

    # тест сортировки слиянием
    temp_list = test_list.copy()
    merge_sort(temp_list)
    assert temp_list == sorted_list

    # тест быстрой сортировки
    temp_list = test_list.copy()
    quicksort_shell(temp_list)
    assert temp_list == sorted_list

    # запуск программы
    main()
